# Agents o Assistants

Sabemos que los LLMs son grandes cajas negras que interpretan el lenguaje natural y nos pueden dar respuestas tambien en nuestro lenguaje de acuerdo a la informacion que tienen comprimida adentro de acuerdo a su entrenamiento.

Que pasa si a estos LLMs ademas les damos herramientas para que puedan obtener informacion actualizada y verificada del mundo real?

## OpenAI Functions and LangChain Tools

Veamos como podemos darles estas herramientas a los LLMs. En OpenAI les llaman functions y en LangChain Tools:

In [ ]:
!pip install openai --upgrade --quiet
!pip install langchain --upgrade --quiet

In [ ]:
import openai

api_key="INGRESA AQUI TU TOKEN DE OPENAI"

In [ ]:
#@title [Opcional] cargar el token desde gdrive para que no se vea al dar clases {display-mode:"form"}
if api_key == "INGRESA AQUI TU TOKEN DE OPENAI":
    from google.colab import drive
    drive.mount('/content/drive')
    with open('drive/MyDrive/Tokens/openai.txt','r') as f:
        token = f.read()
        openai.api_key = token

### OpenAI functions:

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

## Creando nuestra propia tool en LangChain

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
agent = initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")

## LangChain Agents

In [ ]:
!pip install -U wikipedia
!pip install openai --upgrade --quiet
!pip install langchain --upgrade --quiet

In [ ]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(openai_api_key=token, temperature=0.0, model="gpt-3.5-turbo")

In [ ]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [ ]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

#### Agente de matematica

In [ ]:
agent("What is the 25% of 300?")

#### Agente de wikipedia

In [ ]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 

#### Agente de python

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

Mas tipos de agentes: https://python.langchain.com/docs/modules/agents/agent_types/

## Crear nuestro propio Agente

- https://blog.langchain.dev/custom-agents/
- https://python.langchain.com/docs/modules/agents/how_to/custom_agent
- https://artificialcorner.com/how-to-build-tool-using-agents-with-langchain-d9fa9674b6c7

## Referencias:
Esta unidad esta basada en el [curso de Andrew Ng de Deeplearning.ai](https://www.deeplearning.ai/short-courses/langchain-for-llm-application-development/)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)